In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform Pillow matplotlib

In [ ]:
# Restart the runtime
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
PROJECT_ID = "alexbu-gke-dev-d"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [6]:
# import libraries
from vertexai.preview.vision_models import ImageGenerationModel

In [8]:
import typing

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_image(
    image,
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)
    if pil_image.mode != "RGB":
        # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
        pil_image = pil_image.convert("RGB")
    image_width, image_height = pil_image.size
    if max_width < image_width or max_height < image_height:
        # Resize to display a smaller notebook image
        pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
    IPython.display.display(pil_image)

In [ ]:
! gcloud auth application-default login

In [ ]:
generation_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")
generation_model_fast = ImageGenerationModel.from_pretrained(
    "imagen-3.0-fast-generate-001"
)

In [ ]:
import matplotlib.pyplot as plt

prompt = """
a photorealistic image of the inside of an amethyst crystal on display in a museum
"""

# Imagen 3 image generation
image = generation_model.generate_images(
    prompt=prompt,
    number_of_images=1,
    aspect_ratio="3:4",
    safety_filter_level="block_some",
    person_generation="allow_adult",
)

# Imagen 3 Fast image generation
fast_image = generation_model_fast.generate_images(
    prompt=prompt,
    number_of_images=1,
    aspect_ratio="3:4",
    safety_filter_level="block_some",
    person_generation="allow_adult",
)

# Display generated images
fig, axis = plt.subplots(1, 2, figsize=(12, 6))
axis[0].imshow(image[0]._pil_image)
axis[0].set_title("Imagen 3")
axis[1].imshow(fast_image[0]._pil_image)
axis[1].set_title("Imagen 3 Fast")
for ax in axis:
    ax.axis("off")
plt.show()

This is the code to generate the prompt:

```python
"""
Install the Google AI Python SDK

$ pip install google-generativeai
"""

import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "Generate a prompt that I can pass to image generation model, like \"imagen\" to generate an image of hint to display next to Chinese word \"包\" pinyin \"bāo\" translated as \"package\" in a language learning app. The hint should be subtle without revealing the image. Generate the prompt text only.",
      ],
    },
    {
      "role": "model",
      "parts": [
        "A brown paper package tied up with string, but zoomed in extremely close on the knotted string so the contents are obscured and only the texture of the string and paper are visible.  Add a subtle warm, inviting light. \n",
      ],
    },
  ]
)

response = chat_session.send_message("INSERT_INPUT_HERE")

print(response.text)

```

